In [450]:
import numpy as np

# test data
t_A = '[&R] ((1:0.96,3:0.96):0.14,2:1.10,4:1.10);' # rooted
t_B = '[&R] (1:1.167,2:1.167,3:1.167:4:1.167);'

t_C = '[&R] ((1:0.5,2:0.5):0.60,(3:0.5,4:0.5):0.60);' # no branches

# species by 3-site matrix
msa = np.array([[1,0,2], # species 1
                 [1,0,0], # species 2
                 [0,1,2], # species 3
                 [0,1,0]] # species 4
               )


In [451]:
# https://bio.libretexts.org/Bookshelves/Evolutionary_Developmental_Biology/Phylogenetic_Comparative_Methods_(Harmon)/08%3A_Fitting_Models_of_Discrete_Character_Evolution/8.07%3A_Appendix_-_Felsenstein's_Pruning_Algorithm
# t = '((((1:1.0, 2:1.0):0.5, 3:1.5):1.0,((4:0.5, 5:0.5):2.0):0.5), 6:2.5);'
# msa = np.array([[0], [1], [0], [2], [2], [1]])


In [452]:

numsites = len(msa[0])
lambda_site = dict()

for site in range(numsites):
    lambda_site[site] = np.unique(msa.T[site])
    
    
char_probs = dict()
for site in range(numsites):
    char_probs[site] = dict()
    chars, counts = np.unique(msa.T[site], return_counts=True)
    for i in range(len(chars)):
        char_probs[site][chars[i]] = counts[i]/len(msa.T[site])

In [453]:
for site in range(numsites):
    print("site:", site)
    print(char_probs[0])

site: 0
{0: 0.5, 1: 0.5}
site: 1
{0: 0.5, 1: 0.5}
site: 2
{0: 0.5, 1: 0.5}


In [454]:
# nwkt.as_string(schema="newick")
# nwkt.as_ascii_plot()

In [455]:
def get_branchlen(child_node):
    # print("getting branch length", child_node.edge_length)
    if child_node.edge_length is None:
        print(child_node.child_nodes())
    return child_node.edge_length

In [456]:
import math

def prob_same(nodedict, node_likelihood, site, curr_node):
    # print("prob_same")
    # prob of staying in 0 
    all_child_prob = 1.0
    # print(curr_node.child_nodes())
    for c in curr_node.child_nodes():
        # print(c)
        # if c.is_internal():
        #     print("c is internal", c)
        # prob_over_child_states
        char_state_prob = 0.0
        for alpha in nodedict[c][site]:
        
            # lambda_i = np.count_nonzero(msa.T[site])/len(msa.T[site])
            # tp = transition[alpha][alpha]
            
            # print("PS: lambda_i", lambda_i)
            print("PS: get_branchlen(c)", c, get_branchlen(c))
            
            tp = math.exp(get_branchlen(c)) # put lambda_i 
            # print("tp", tp)
            char_state_prob += tp * node_likelihood[c][site]
            # print("char_state_prob", char_state_prob)
        all_child_prob *= char_state_prob
    # print("all_child_prob", all_child_prob)        
    return all_child_prob

def prob_change(nodedict, node_likelihood, site, curr_state, curr_node):
    # print("prob_change")
    # prob of changing from 0 to alpha 
    all_child_prob = 1.0
    # print(curr_node.child_nodes())
    for c in curr_node.child_nodes():
        # print(c)
        # if c.is_internal():
        #     print("c is internal", c)
        char_state_prob = 0.0
        for alpha in nodedict[c][site]:
            # lambda_i, total number of changes at site i
            lambda_i = np.count_nonzero(msa.T[site])/len(msa.T[site])
            # lambda_i_alpha, total number of changes to alpha at site i
            lambda_i_alpha = np.count_nonzero(msa.T[site] == curr_state)/len(msa.T[site])
            
            tp = lambda_i_alpha/lambda_i * (1 - math.exp(-lambda_i * get_branchlen(c)))
            # print("PC: ratio", lambda_i_alpha/lambda_i)
            # print("PC: tp", tp)
            # print("PC: node likelihood", node_likelihood[c][site])
            char_state_prob += tp * node_likelihood[c][site]
            # print("char_state_prob", char_state_prob)
        all_child_prob *= char_state_prob
    # print("PC: all_child_prob", all_child_prob)
    return all_child_prob


In [457]:
def likelihood_under_n(nodedict, node_likelihood, n, site):
    # n is an internal node
    # for node n, get children's possible states from nodedict
    
    child_states = set()
        
    if n not in nodedict:
        nodedict[n] = dict()
        nodedict[n][site] = dict()
        
    # identify all child states. 
    # this constrains n's possible states.
    child_states = set()
    for child in n.child_nodes():
        # print("child", child, child.is_leaf(), child.is_internal())
        if child.is_leaf():
            child_states.add(get_char(msa, child, site))
        else:
            for x in nodedict[child][site]:
                child_states.add(x)
    # print("child_states", child_states)
    possible_states = dict()
    if len(child_states) == 1:
        if 0 in child_states:
            # probability 0 -> 0
            possible_states[0] = prob_same(nodedict, node_likelihood, site, n) 
        else:
            for c in child_states:
                # probability c -> c != 0
                possible_states[c] = 1.0 
            # probability 0 -> c (alpha)
            possible_states[0] = prob_change(nodedict, node_likelihood, site, 0, n)  
    else:
        # probability 0 -> 1 and 0 -> 2 or
        # probability 0 -> 0 and 0 -> 1 WLOG
        possible_states[0] = 1.0

    for x in possible_states.keys():
        # save into nodedict
        nodedict[n][site][x] = possible_states[x]
        # product over all possible states
        node_likelihood[n][site] *= possible_states[x]
    
    return nodedict, node_likelihood

In [458]:
def get_char(msa, leaf_node, site):
    # print(leaf_node)
    return msa[int(leaf_node.taxon.__str__().replace("'", ""))-1][site]

In [459]:
import dendropy

nwkt = dendropy.Tree.get(data=t_C, schema="newick")
print(nwkt)
nodedict = dict() # maps node to possible states, with probabilities
node_likelihood = dict() # maps node to likelihood of subtree under node

for n in nwkt.postorder_node_iter():
    # print("node:", n)
    if n.taxon is not None: # must be a leaf node, set up 
        nodedict[n] = dict()
        node_likelihood[n] = dict()
        for site in range(numsites):
            char_state = get_char(msa, n, site)
            nodedict[n][site] = dict()
            nodedict[n][site][char_state] = 1.0
            node_likelihood[n][site] = 1.0
        
    elif n.taxon is None: # must be an internal node
        # print("Child Nodes:", n.child_nodes())
        for site in range(numsites):
            # print("site:", site)
            if n not in nodedict:
                nodedict[n] = dict()
                node_likelihood[n] = dict()
            
            # print("PRE")
            # print("NodeDict:", nodedict)
            # print("NodeLikelihood:", node_likelihood)
            
            nodedict[n][site] = dict()
            node_likelihood[n][site] = 1.0
            
            nodedict, node_likelihood = likelihood_under_n(nodedict, node_likelihood, n, site)
            
            # print("POST")
            # print("NodeDict:", nodedict)
            # print("NodeLikelihood:", node_likelihood[n])

tree_likelihood = 1.0
# print likelihood for the root
for child in nwkt.seed_node.child_nodes():
    for site in range(numsites):
        for char in nodedict[child][site]:
            # only have one child, add a r*, the q should also appear
            # don't use the stationary at the root, instead we want to use the distance 1 - e^...
            # initialize with some distance 
            
            tree_likelihood *= char_probs[site][char] * node_likelihood[child][site]
            
print(tree_likelihood)



((1:0.5,2:0.5):0.6,(3:0.5,4:0.5):0.6)
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47650: 'None' (<Taxon 0x7f8734e47fd0 '1'>)> 0.5
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47490: 'None' (<Taxon 0x7f8734e47a10 '2'>)> 0.5
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47d50: 'None' (<Taxon 0x7f8734e47150 '3'>)> 0.5
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47ed0: 'None' (<Taxon 0x7f8734e47610 '4'>)> 0.5
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47f10: 'None' (None)> 0.6
PS: lambda_i 0.5
PS: get_branchlen(c) <Node object at 0x7f8734e47990: 'None' (None)> 0.6
8.236339393882161e-09


In [460]:
t_A = 2.062468362106625e-05
t_B = 0.001953125
t_C = 8.236339393882161e-09

In [461]:
t_C < t_B

True

In [462]:
t_A > t_C

True

In [ ]:
# ziheng yang - must always be less than f_max <= 
# as you increase the number of k, should converge, check paper... 

In [ ]:
# make this into a function 
def felsenstein(T, Q, msa): 
    # tree w/ branch lengths as input
    # output a likelihood
    ##
    pass 
